In [1]:
using Pkg; Pkg.activate("."); Pkg.instantiate();

using Flux
using Flux: onehot, chunk, batchseq, throttle, crossentropy
using StatsBase: wsample
using Base.Iterators: partition
using CuArrays

cd(@__DIR__)

isfile("input.txt") ||
  download("https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt",
           "input.txt")

text = collect(String(read("input.txt")))
alphabet = [unique(text)..., '_']
text = map(ch -> onehot(ch, alphabet), text)
stop = onehot('_', alphabet)

N = length(alphabet)
seqlen = 50
nbatch = 50

Xs = collect(partition(batchseq(chunk(text, nbatch), stop), seqlen))
Ys = collect(partition(batchseq(chunk(text[2:end], nbatch), stop), seqlen))

m = Chain(
  LSTM(N, 128),
  LSTM(128, 128),
  Dense(128, N),
  softmax)

m = gpu(m)

function loss(xs, ys)
  l = sum(crossentropy.(m.(gpu.(xs)), gpu.(ys)))
  Flux.truncate!(m)
  return l
end

opt = ADAM(params(m), 0.01)
tx, ty = (gpu.(Xs[5]), gpu.(Ys[5]))
evalcb = () -> @show loss(tx, ty)

Flux.train!(loss, zip(Xs, Ys), opt,
            cb = throttle(evalcb, 30))
# Sampling
m = cpu(m)

function sample(m, alphabet, len; temp = 1)
  Flux.reset!(m)
  buf = IOBuffer()
  c = rand(alphabet)
  for i = 1:len
    write(buf, c)
    c = wsample(alphabet, m(onehot(c, alphabet)).data)
  end
  return String(take!(buf))
end

sample(m, alphabet, 1000) |> println

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
┌ Warning: Error requiring CuArrays from Flux:
│ LoadError: UndefVarError: cudnn_available not defined
│ Stacktrace:
│  [1] getproperty(::Module, ::Symbol) at ./sysimg.jl:13
│  [2] top-level scope at none:0
│  [3] include at ./boot.jl:317 [inlined]
│  [4] include_relative(::Module, ::String) at ./loading.jl:1038
│  [5] include at ./sysimg.jl:29 [inlined]
│  [6] include(::String) at /home/dhairyagandhi96/.julia/packages/Flux/UHjNa/src/Flux.jl:1
│  [7] top-level scope at none:0
│  [8] eval at ./boot.jl:319 [inlined]
│  [9] eval at /home/dhairyagandhi96/.julia/packages/Flux/UHjNa/src/Flux.jl:1 [inlined]
│  [10] (::getfield(Flux, Symbol("##99#103")))() at /home/dhairyagandhi96/.julia/packages/Requires/9Jse/src/require.jl:67
│  [11] err(::getfield(Flux, Symbol("##99#103")), ::Module, ::String) at /home/dhairyagandhi96/.julia/packages/Requires/9Jse/src/require.jl:38
│  [1

evalcb = function ()
  @show loss(Xs[5], Ys[5])
  println(sample(deepcopy(m), alphabet, 100))
end